# <font color='red'> Time Series Modelling </font>

To predict the future values for stock markets, electricity demand, pollution levels, etc we need to enter into the realm of **forecasting**. 

Let's go through some timeseries basics

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})

import pandas as pd
import numpy as np

## <font color='red'> Modelling Basics </font>

Throughout time series literature, you will find several terms like **trend**, **seasonal**, **exogenous variable**, etc. These terms are used to refer to specific components of a time series and what (external) phenomena might be connected to the what you are trying to predict. Let's start with the components and, after that, the nomenclature for variables.

### <font color='red'> 1. Components <font>

Time series can be thought as a (linear or non-linear) composition of 4 components: **trend**, **cyclical**, **seasonal** and **irregular**

$$Y_t = Trend + Cyclical + Seasonal + Irregular$$

Or

$$Y_t = Trend \cdot Cyclical \cdot Seasonal \cdot Irregular$$

Or another non-linear combination of all four. Each one of the previous four components is also time series.

### 1.1. Trend

The trend is the component of the time series that allows us to see if, in general, the dependent variable we are observing is increasing, without taking into consideration local flutuations. Usually, people look at the trend to see if the mean value of a series is (monotonically) increasing or decreasing. The trend can be modelled as a linear or non-linear process, even though people prefer to assume it is linear.

In order to understand the concept of trend, let's look at a small dataset with [monthly shampoo sales for a shampoo company over a 3 year period](https://www.kaggle.com/dougcresswell/shampoo-sales-2001-2003).

We have a file `utils.py` that loads the data instead of putting all that code here

In [ ]:
from utils import load_shampoo_series

shampoo_sales = load_shampoo_series()

shampoo_sales.plot(figsize=(10, 8));

The first thing you probably notice is that, in general, the *sales are increasing* even though that are anual peaks. In order to visualize the trend, let's plot a simple linear regression that maps the time and the sales

In [ ]:
from utils import show_shampoo_sales_and_trend

coef = show_shampoo_sales_and_trend(figsize=(10, 8))

print("This trend tells us that, between any two consecutive months, the overall sales increase by {}.".format(coef))

Having a non-zero trend like the one we got in here is not a good thing when using certain models. We will learn more about it in the next notebooks.

### 1.2. Cyclical 

The cyclical component is, as the name states, a time series that exhibits a behavior that repeats in non-regular time intervals. For example, the performance of the world economy in the 20st century would exhibit a strong cyclical component with non-regular cycles (remember the crisis at the 80's and 2007). This is longer than a year.

###  1.3. Seasonal 

Unlike the cyclical component, the seasonal component changes at a fixed rate. For example, the bookings in hotels in certain cities have well known maxima (Autumn) and minima (Summer). Another clear example of seasonality is the one we have in a famous dataset that shows the [number of internaltional airline passengers (in thousands) by year](https://www.kaggle.com/andreazzini/international-airline-passengers), spanning from 1949 to 1961

In [ ]:
from utils import load_airlines_series

airlines = load_airlines_series()

airlines.head(10)

In [ ]:
airlines.plot();

In this time series, we can see, at least, two things: (1) a positive trend and (2) yearly minima and maxima approximately in the same months. To make (2) more clear, let's plot the time series for each year

In [ ]:
from utils import plot_seasonality_for_airlines

In [ ]:
plot_seasonality_for_airlines(normalize=False);

Around June-July, we get the peak and around October, we get the period with less passengers. In order for this pattern to be clear for you, let's normalize the season time series for each year

In [ ]:
plot_seasonality_for_airlines(normalize=True);

from the normalized plot, we can see a clear pattern in the second half of each year but we get a more dubious pattern for the first half.

### 1.4. Irregular

After accounting for all the previous components, the remaining component, called *irregular* or *residual*, won't have any pattern. This part of the time series is considered noise. You might be thinking that this component is useless. But, in fact, several modelling techniques analyze this component in order to check a better model can be created.

<hr>


## What do these look like combined?
After introducing all 4 components, let's look into more examples

![four example time series](media/ts-examples.png)

- The upper left time series shows both a seasonal component for each year and a strong cyclical component that takes 6-10 years (imagine an ark connecting 1975 to 1981 and another one connecting 1981 and 1991) but no apparent trend.

- In the upper right corner, we have a strong negative trend but no visible seasonal nor cyclical behavior. This might be due to how small the time series was. 

- In the lower left corner, shows both strong (positive trend) and seasonality but no cyclical component.

- Finally, the time series at the lower right corner looks like pure noise, with some peaks. No clear pattern.


You might be asking "Are there any tools to help me identify all 4 components? After identifying them, what can I do with them?".

-------------

## <font color='red'> Seasonal Decomposition </font>

We just saw two ways of combining the time series components: additive ($Y = T + C + S + I$) and multiplicative ($Y = T \cdot C \cdot S \cdot I$). But...how do we know if our time series is additive or multiplicative? Citing [Hyndman & Athanasopoulos](https://otexts.org/fpp2/components.html)

> _The additive decomposition is the most appropriate if the magnitude of the seasonal fluctuations, or the **variation around the trend-cycle**, does not vary with the level of the time series. When the variation in the seasonal pattern, or the variation around the trend-cycle, appears to be proportional to the level of the time series, then a multiplicative decomposition is more appropriate. Multiplicative decompositions are common with economic time series._

There are several ways to implement this: using [moving averages](https://otexts.org/fpp2/classical-decomposition.html), [X11](https://otexts.org/fpp2/x11.html), [SEATS](https://otexts.org/fpp2/seats.html) and [STL](https://otexts.org/fpp2/stl.html). Here we will use the **moving averages** approach from [statsmodel](https://www.statsmodels.org/stable/index.html)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
ts = load_airlines_series()

ts.head()

We need to choose if we will use `model=additive` or `model=multiplicative`. In order to do that, we have to look at our time series and follow the Hyndman & Athanasopoulos guidelines

In [ ]:
ts.plot();

It seems that the variance around the trend (remember the linear trend estimation?) is increasing throughout time. To check that visually, let's use the `pandas.Series.rolling.std` and plot the result

In [ ]:
ts.rolling(12).std().plot();

using the previous plot, we can confirm that the variance of this time series increases throughout time. As such, let's use the multiplicative model for decomposition

In [ ]:
decomposition = seasonal_decompose(ts, model='multiplicative')
decomposition

with this object, we have access to the estimation of the following components: 
* Trend-Cyclical
* Seasonal
* Irregular

statsmodel includes a function to plot all 3 components at once

In [ ]:
decomposition.plot();

we have direct access to all 3 components

In [ ]:
decomposition.trend.head(10)

In [ ]:
decomposition.seasonal.head(10)

In [ ]:
decomposition.resid.head(10)

We can reconstruct the original ts by multiplying its components

In [ ]:
ts.plot(label="original", linestyle ="dashed")
(decomposition.trend * decomposition.resid * decomposition.seasonal).plot(label="recomposed", linestyle="dashed")
sns.mpl.pyplot.legend();